In [124]:
from bs4 import BeautifulSoup
import urllib.request as url
import re

def find_player_id():
    players = {}
    player_pages = ["http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false", "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/41", "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/81", "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/121", "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/121", "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/161","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/201","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/241","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/281","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/321","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/361","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/401","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/441"]
    for page in player_pages:
        html_page = url.urlopen(page)
        soup = BeautifulSoup(html_page)
        for link in soup.findAll('a', attrs={'href': re.compile("^http://www.espn.com/nba/player/_/id")}):
            players_id = link.get('href').split('/')
            players[players_id[len(players_id)-1]] = players_id[len(players_id)-2]
    return players;    

def find_teams_url(team_url):
    html_page = url.urlopen(team_url)
    soup = BeautifulSoup(html_page)
    team_url = []
    for link in soup.findAll('a', attrs={'href': re.compile("^http://www.espn.com/nba/team/_/name/")}):
        team_url.append(link.get('href'))
    return set(team_url)

In [112]:
import pandas
def read_players(player_id, years):
    player_info = {}
    for player in player_id:
        player_years = {}
        for year in years:
            url_path = "http://www.espn.com/nba/player/gamelog/_/id/" + player_id[player] + "/year/" + str(year) + "/" + player
            try:
                dataframe = pandas.read_html(url_path)
                player_years[year] = dataframe
            except url.HTTPError:
                pass
            print(url_path)
        player_info[player] = player_years
    return player_info

In [101]:
player_id = find_player_id()
test = read_players(player_id,[2018])

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


http://www.espn.com/nba/player/gamelog/_/id/3975/year/2018/stephen-curry
http://www.espn.com/nba/player/gamelog/_/id/3992/year/2018/james-harden
http://www.espn.com/nba/player/gamelog/_/id/3202/year/2018/kevin-durant
http://www.espn.com/nba/player/gamelog/_/id/1966/year/2018/lebron-james
http://www.espn.com/nba/player/gamelog/_/id/6583/year/2018/anthony-davis
http://www.espn.com/nba/player/gamelog/_/id/6606/year/2018/damian-lillard
http://www.espn.com/nba/player/gamelog/_/id/3032977/year/2018/giannis-antetokounmpo
http://www.espn.com/nba/player/gamelog/_/id/3059318/year/2018/joel-embiid
http://www.espn.com/nba/player/gamelog/_/id/6450/year/2018/kawhi-leonard
http://www.espn.com/nba/player/gamelog/_/id/6479/year/2018/kemba-walker
http://www.espn.com/nba/player/gamelog/_/id/3989/year/2018/blake-griffin
http://www.espn.com/nba/player/gamelog/_/id/3064440/year/2018/zach-lavine
http://www.espn.com/nba/player/gamelog/_/id/3978/year/2018/demar-derozan
http://www.espn.com/nba/player/gamelog/_/

http://www.espn.com/nba/player/gamelog/_/id/2284101/year/2018/justin-holiday
http://www.espn.com/nba/player/gamelog/_/id/6581/year/2018/jae-crowder
http://www.espn.com/nba/player/gamelog/_/id/3213/year/2018/al-horford
http://www.espn.com/nba/player/gamelog/_/id/4066328/year/2018/jarrett-allen
http://www.espn.com/nba/player/gamelog/_/id/3133603/year/2018/kelly-oubre-jr.
http://www.espn.com/nba/player/gamelog/_/id/4015/year/2018/jeff-teague
http://www.espn.com/nba/player/gamelog/_/id/2579260/year/2018/trey-burke
http://www.espn.com/nba/player/gamelog/_/id/2594922/year/2018/otto-porter-jr.
http://www.espn.com/nba/player/gamelog/_/id/3892818/year/2018/emmanuel-mudiay
http://www.espn.com/nba/player/gamelog/_/id/3452/year/2018/javale-mcgee
http://www.espn.com/nba/player/gamelog/_/id/6461/year/2018/markieff-morris
http://www.espn.com/nba/player/gamelog/_/id/6474/year/2018/tristan-thompson
http://www.espn.com/nba/player/gamelog/_/id/2994526/year/2018/bryn-forbes
http://www.espn.com/nba/player/

http://www.espn.com/nba/player/gamelog/_/id/6446/year/2018/cory-joseph
http://www.espn.com/nba/player/gamelog/_/id/3078286/year/2018/troy-williams
http://www.espn.com/nba/player/gamelog/_/id/4244/year/2018/lance-stephenson
http://www.espn.com/nba/player/gamelog/_/id/3998/year/2018/jonas-jerebko
http://www.espn.com/nba/player/gamelog/_/id/6428/year/2018/marshon-brooks
http://www.espn.com/nba/player/gamelog/_/id/4230550/year/2018/elie-okobo
http://www.espn.com/nba/player/gamelog/_/id/3999/year/2018/james-johnson
http://www.espn.com/nba/player/gamelog/_/id/2761/year/2018/gerald-green
http://www.espn.com/nba/player/gamelog/_/id/2991042/year/2018/nik-stauskas
http://www.espn.com/nba/player/gamelog/_/id/2999409/year/2018/willy-hernangomez
http://www.espn.com/nba/player/gamelog/_/id/6617/year/2018/austin-rivers
http://www.espn.com/nba/player/gamelog/_/id/2579466/year/2018/jonathon-simmons
http://www.espn.com/nba/player/gamelog/_/id/2011/year/2018/kyle-korver
http://www.espn.com/nba/player/gam

http://www.espn.com/nba/player/gamelog/_/id/3444/year/2018/kosta-koufos
http://www.espn.com/nba/player/gamelog/_/id/3948153/year/2018/chris-boucher
http://www.espn.com/nba/player/gamelog/_/id/2596108/year/2018/michael-carter-williams
http://www.espn.com/nba/player/gamelog/_/id/4065651/year/2018/frank-jackson
http://www.espn.com/nba/player/gamelog/_/id/2531047/year/2018/jerian-grant
http://www.espn.com/nba/player/gamelog/_/id/4230546/year/2018/terrance-ferguson
http://www.espn.com/nba/player/gamelog/_/id/2326307/year/2018/seth-curry
http://www.espn.com/nba/player/gamelog/_/id/2769/year/2018/amir-johnson
http://www.espn.com/nba/player/gamelog/_/id/3058254/year/2018/christian-wood
http://www.espn.com/nba/player/gamelog/_/id/3974/year/2018/dante-cunningham
http://www.espn.com/nba/player/gamelog/_/id/3919335/year/2018/cheick-diallo
http://www.espn.com/nba/player/gamelog/_/id/2016/year/2018/zaza-pachulia
http://www.espn.com/nba/player/gamelog/_/id/2489785/year/2018/ian-clark
http://www.espn.

http://www.espn.com/nba/player/gamelog/_/id/4067045/year/2018/alize-johnson
http://www.espn.com/nba/player/gamelog/_/id/2531797/year/2018/jordan-mcrae
http://www.espn.com/nba/player/gamelog/_/id/3934621/year/2018/jacob-evans
http://www.espn.com/nba/player/gamelog/_/id/2282117/year/2018/john-holland
http://www.espn.com/nba/player/gamelog/_/id/2528473/year/2018/okaro-white
http://www.espn.com/nba/player/gamelog/_/id/2595231/year/2018/treveon-graham
http://www.espn.com/nba/player/gamelog/_/id/2982240/year/2018/jaron-blossomgame
http://www.espn.com/nba/player/gamelog/_/id/2990968/year/2018/naz-mitrou-long
http://www.espn.com/nba/player/gamelog/_/id/3059262/year/2018/davon-reed
http://www.espn.com/nba/player/gamelog/_/id/3059430/year/2018/zach-lofton
http://www.espn.com/nba/player/gamelog/_/id/3064560/year/2018/luke-kornet
http://www.espn.com/nba/player/gamelog/_/id/3136194/year/2018/tyler-ulis
http://www.espn.com/nba/player/gamelog/_/id/3908806/year/2018/ray-spalding
http://www.espn.com/nb

In [52]:
train = read_players(player_id, [2013,2014,2015,2016,2017])

http://www.espn.com/nba/player/gamelog/_/id/3975/year/2013/stephen-curry
http://www.espn.com/nba/player/gamelog/_/id/3975/year/2014/stephen-curry
http://www.espn.com/nba/player/gamelog/_/id/3975/year/2015/stephen-curry
http://www.espn.com/nba/player/gamelog/_/id/3975/year/2016/stephen-curry
http://www.espn.com/nba/player/gamelog/_/id/3975/year/2017/stephen-curry
http://www.espn.com/nba/player/gamelog/_/id/3992/year/2013/james-harden
http://www.espn.com/nba/player/gamelog/_/id/3992/year/2014/james-harden
http://www.espn.com/nba/player/gamelog/_/id/3992/year/2015/james-harden
http://www.espn.com/nba/player/gamelog/_/id/3992/year/2016/james-harden
http://www.espn.com/nba/player/gamelog/_/id/3992/year/2017/james-harden
http://www.espn.com/nba/player/gamelog/_/id/3202/year/2013/kevin-durant
http://www.espn.com/nba/player/gamelog/_/id/3202/year/2014/kevin-durant
http://www.espn.com/nba/player/gamelog/_/id/3202/year/2015/kevin-durant
http://www.espn.com/nba/player/gamelog/_/id/3202/year/2016/

http://www.espn.com/nba/player/gamelog/_/id/3136195/year/2014/karl-anthony-towns
http://www.espn.com/nba/player/gamelog/_/id/3136195/year/2015/karl-anthony-towns
http://www.espn.com/nba/player/gamelog/_/id/3136195/year/2016/karl-anthony-towns
http://www.espn.com/nba/player/gamelog/_/id/3136195/year/2017/karl-anthony-towns
http://www.espn.com/nba/player/gamelog/_/id/4237/year/2013/john-wall
http://www.espn.com/nba/player/gamelog/_/id/4237/year/2014/john-wall
http://www.espn.com/nba/player/gamelog/_/id/4237/year/2015/john-wall
http://www.espn.com/nba/player/gamelog/_/id/4237/year/2016/john-wall
http://www.espn.com/nba/player/gamelog/_/id/4237/year/2017/john-wall
http://www.espn.com/nba/player/gamelog/_/id/3195/year/2013/mike-conley
http://www.espn.com/nba/player/gamelog/_/id/3195/year/2014/mike-conley
http://www.espn.com/nba/player/gamelog/_/id/3195/year/2015/mike-conley
http://www.espn.com/nba/player/gamelog/_/id/3195/year/2016/mike-conley
http://www.espn.com/nba/player/gamelog/_/id/319

http://www.espn.com/nba/player/gamelog/_/id/3936299/year/2015/jamal-murray
http://www.espn.com/nba/player/gamelog/_/id/3936299/year/2016/jamal-murray
http://www.espn.com/nba/player/gamelog/_/id/3936299/year/2017/jamal-murray
http://www.espn.com/nba/player/gamelog/_/id/3102529/year/2013/clint-capela
http://www.espn.com/nba/player/gamelog/_/id/3102529/year/2014/clint-capela
http://www.espn.com/nba/player/gamelog/_/id/3102529/year/2015/clint-capela
http://www.espn.com/nba/player/gamelog/_/id/3102529/year/2016/clint-capela
http://www.espn.com/nba/player/gamelog/_/id/3102529/year/2017/clint-capela
http://www.espn.com/nba/player/gamelog/_/id/6578/year/2013/harrison-barnes
http://www.espn.com/nba/player/gamelog/_/id/6578/year/2014/harrison-barnes
http://www.espn.com/nba/player/gamelog/_/id/6578/year/2015/harrison-barnes
http://www.espn.com/nba/player/gamelog/_/id/6578/year/2016/harrison-barnes
http://www.espn.com/nba/player/gamelog/_/id/6578/year/2017/harrison-barnes
http://www.espn.com/nba/p

http://www.espn.com/nba/player/gamelog/_/id/3593/year/2016/bojan-bogdanovic
http://www.espn.com/nba/player/gamelog/_/id/3593/year/2017/bojan-bogdanovic
http://www.espn.com/nba/player/gamelog/_/id/6443/year/2013/reggie-jackson
http://www.espn.com/nba/player/gamelog/_/id/6443/year/2014/reggie-jackson
http://www.espn.com/nba/player/gamelog/_/id/6443/year/2015/reggie-jackson
http://www.espn.com/nba/player/gamelog/_/id/6443/year/2016/reggie-jackson
http://www.espn.com/nba/player/gamelog/_/id/6443/year/2017/reggie-jackson
http://www.espn.com/nba/player/gamelog/_/id/3431/year/2013/eric-gordon
http://www.espn.com/nba/player/gamelog/_/id/3431/year/2014/eric-gordon
http://www.espn.com/nba/player/gamelog/_/id/3431/year/2015/eric-gordon
http://www.espn.com/nba/player/gamelog/_/id/3431/year/2016/eric-gordon
http://www.espn.com/nba/player/gamelog/_/id/3431/year/2017/eric-gordon
http://www.espn.com/nba/player/gamelog/_/id/4278129/year/2013/deandre-ayton
http://www.espn.com/nba/player/gamelog/_/id/427

http://www.espn.com/nba/player/gamelog/_/id/3102530/year/2016/jusuf-nurkic
http://www.espn.com/nba/player/gamelog/_/id/3102530/year/2017/jusuf-nurkic
http://www.espn.com/nba/player/gamelog/_/id/3012/year/2013/kyle-lowry
http://www.espn.com/nba/player/gamelog/_/id/3012/year/2014/kyle-lowry
http://www.espn.com/nba/player/gamelog/_/id/3012/year/2015/kyle-lowry
http://www.espn.com/nba/player/gamelog/_/id/3012/year/2016/kyle-lowry
http://www.espn.com/nba/player/gamelog/_/id/3012/year/2017/kyle-lowry
http://www.espn.com/nba/player/gamelog/_/id/6619/year/2013/terrence-ross
http://www.espn.com/nba/player/gamelog/_/id/6619/year/2014/terrence-ross
http://www.espn.com/nba/player/gamelog/_/id/6619/year/2015/terrence-ross
http://www.espn.com/nba/player/gamelog/_/id/6619/year/2016/terrence-ross
http://www.espn.com/nba/player/gamelog/_/id/6619/year/2017/terrence-ross
http://www.espn.com/nba/player/gamelog/_/id/3149673/year/2013/pascal-siakam
http://www.espn.com/nba/player/gamelog/_/id/3149673/year/20

...

http://www.espn.com/nba/player/gamelog/_/id/3913174/year/2015/luke-kennard
http://www.espn.com/nba/player/gamelog/_/id/3913174/year/2016/luke-kennard
http://www.espn.com/nba/player/gamelog/_/id/3913174/year/2017/luke-kennard
http://www.espn.com/nba/player/gamelog/_/id/2528693/year/2013/torrey-craig
http://www.espn.com/nba/player/gamelog/_/id/2528693/year/2014/torrey-craig
http://www.espn.com/nba/player/gamelog/_/id/2528693/year/2015/torrey-craig
http://www.espn.com/nba/player/gamelog/_/id/2528693/year/2016/torrey-craig
http://www.espn.com/nba/player/gamelog/_/id/2528693/year/2017/torrey-craig
http://www.espn.com/nba/player/gamelog/_/id/2774/year/2013/ian-mahinmi
http://www.espn.com/nba/player/gamelog/_/id/2774/year/2014/ian-mahinmi
http://www.espn.com/nba/player/gamelog/_/id/2774/year/2015/ian-mahinmi
http://www.espn.com/nba/player/gamelog/_/id/2774/year/2016/ian-mahinmi
http://www.espn.com/nba/player/gamelog/_/id/2774/year/2017/ian-mahinmi
http://www.espn.com/nba/player/gamelog/_/id/6

http://www.espn.com/nba/player/gamelog/_/id/3136989/year/2017/vincent-edwards
http://www.espn.com/nba/player/gamelog/_/id/3002137/year/2013/tyrone-wallace
http://www.espn.com/nba/player/gamelog/_/id/3002137/year/2014/tyrone-wallace
http://www.espn.com/nba/player/gamelog/_/id/3002137/year/2015/tyrone-wallace
http://www.espn.com/nba/player/gamelog/_/id/3002137/year/2016/tyrone-wallace
http://www.espn.com/nba/player/gamelog/_/id/3002137/year/2017/tyrone-wallace
http://www.espn.com/nba/player/gamelog/_/id/3943606/year/2013/jerome-robinson
http://www.espn.com/nba/player/gamelog/_/id/3943606/year/2014/jerome-robinson
http://www.espn.com/nba/player/gamelog/_/id/3943606/year/2015/jerome-robinson
http://www.espn.com/nba/player/gamelog/_/id/3943606/year/2016/jerome-robinson
http://www.espn.com/nba/player/gamelog/_/id/3943606/year/2017/jerome-robinson
http://www.espn.com/nba/player/gamelog/_/id/3064517/year/2013/jarell-martin
http://www.espn.com/nba/player/gamelog/_/id/3064517/year/2014/jarell-ma

http://www.espn.com/nba/player/gamelog/_/id/4222252/year/2014/isaiah-hartenstein
http://www.espn.com/nba/player/gamelog/_/id/4222252/year/2015/isaiah-hartenstein
http://www.espn.com/nba/player/gamelog/_/id/4222252/year/2016/isaiah-hartenstein
http://www.espn.com/nba/player/gamelog/_/id/4222252/year/2017/isaiah-hartenstein
http://www.espn.com/nba/player/gamelog/_/id/2578213/year/2013/ben-mclemore
http://www.espn.com/nba/player/gamelog/_/id/2578213/year/2014/ben-mclemore
http://www.espn.com/nba/player/gamelog/_/id/2578213/year/2015/ben-mclemore
http://www.espn.com/nba/player/gamelog/_/id/2578213/year/2016/ben-mclemore
http://www.espn.com/nba/player/gamelog/_/id/2578213/year/2017/ben-mclemore
http://www.espn.com/nba/player/gamelog/_/id/3907487/year/2013/marquese-chriss
http://www.espn.com/nba/player/gamelog/_/id/3907487/year/2014/marquese-chriss
http://www.espn.com/nba/player/gamelog/_/id/3907487/year/2015/marquese-chriss
http://www.espn.com/nba/player/gamelog/_/id/3907487/year/2016/marqu

http://www.espn.com/nba/player/gamelog/_/id/3155533/year/2017/jonah-bolden
http://www.espn.com/nba/player/gamelog/_/id/4261/year/2013/ekpe-udoh
http://www.espn.com/nba/player/gamelog/_/id/4261/year/2014/ekpe-udoh
http://www.espn.com/nba/player/gamelog/_/id/4261/year/2015/ekpe-udoh
http://www.espn.com/nba/player/gamelog/_/id/4261/year/2016/ekpe-udoh
http://www.espn.com/nba/player/gamelog/_/id/4261/year/2017/ekpe-udoh
http://www.espn.com/nba/player/gamelog/_/id/3157465/year/2013/duncan-robinson
http://www.espn.com/nba/player/gamelog/_/id/3157465/year/2014/duncan-robinson
http://www.espn.com/nba/player/gamelog/_/id/3157465/year/2015/duncan-robinson
http://www.espn.com/nba/player/gamelog/_/id/3157465/year/2016/duncan-robinson
http://www.espn.com/nba/player/gamelog/_/id/3157465/year/2017/duncan-robinson
http://www.espn.com/nba/player/gamelog/_/id/3147351/year/2013/daniel-hamilton
http://www.espn.com/nba/player/gamelog/_/id/3147351/year/2014/daniel-hamilton
http://www.espn.com/nba/player/gam

http://www.espn.com/nba/player/gamelog/_/id/4067045/year/2015/alize-johnson
http://www.espn.com/nba/player/gamelog/_/id/4067045/year/2016/alize-johnson
http://www.espn.com/nba/player/gamelog/_/id/4067045/year/2017/alize-johnson
http://www.espn.com/nba/player/gamelog/_/id/2531797/year/2013/jordan-mcrae
http://www.espn.com/nba/player/gamelog/_/id/2531797/year/2014/jordan-mcrae
http://www.espn.com/nba/player/gamelog/_/id/2531797/year/2015/jordan-mcrae
http://www.espn.com/nba/player/gamelog/_/id/2531797/year/2016/jordan-mcrae
http://www.espn.com/nba/player/gamelog/_/id/2531797/year/2017/jordan-mcrae
http://www.espn.com/nba/player/gamelog/_/id/3934621/year/2013/jacob-evans
http://www.espn.com/nba/player/gamelog/_/id/3934621/year/2014/jacob-evans
http://www.espn.com/nba/player/gamelog/_/id/3934621/year/2015/jacob-evans
http://www.espn.com/nba/player/gamelog/_/id/3934621/year/2016/jacob-evans
http://www.espn.com/nba/player/gamelog/_/id/3934621/year/2017/jacob-evans
http://www.espn.com/nba/pla

In [99]:
table = train['jarell-martin'][2017]
print(table[1])

                                    0       1          2     3        4   \
0        2016-2017 POSTSEASON GAME LOG     NaN        NaN   NaN      NaN   
1                                 DATE     OPP      SCORE   MIN  FGM-FGA   
2                             Thu 4/27    vsSA   L 96-103     0      0-0   
3                             Tue 4/25    @ SA  L 103-116     2      1-2   
4                             Sat 4/22    vsSA  W 110-108     0      0-0   
5                             Thu 4/20    vsSA   W 105-94     2      0-0   
6                             Mon 4/17    @ SA    L 82-96     0      0-0   
7                             Sat 4/15    @ SA   L 82-111     6      0-1   
8             Conference Quarterfinals     3.3    0.3-1.0  .333  0.0-0.0   
9                     POSTSEASON STATS     MIN    FGM-FGA   FG%  3PM-3PA   
10                            Averages     3.3    0.3-1.0  .333  0.0-0.0   
11                              Totals      10        1-3  .333      0-0   
12   2016-20

In [216]:
def read_teams(team_url, years):
    team_stat = {}
    for url_path in team_url:
        url_component = url_path.split('/_/')
        stat_url = url_component[0] + '/schedule/_/' + 'name/' + url_component[1].split('/')[1] + '/season/'
        year_stat = {}
        for year in years:
            try:
                dataframe = pandas.read_html(stat_url + str(year) + '/seasontype/2')
                year_stat[year] = dataframe
            except url.HTTPError:
                pass
            print(stat_url + str(year) + '/seasontype/2')
        frames = []
        for year in years:
            df_changed = year_stat[year][2].loc[2:, 0:2]
            before_half = df_changed[0].str.split(" ",expand=True)[1].isin(["Jan","Feb","Mar","Apr","May","Jun","Jul"])
            df_changed[0][before_half] = df_changed[0][before_half]+', '+str(year)
            df_changed[0][~before_half] = df_changed[0][~before_half]+', '+str(year-1)
            frames.append(df_changed)
        result = pandas.concat(frames)
        result.index = range(len(result))
        team_stat[url_component[1][5:]] = result
    return team_stat

In [217]:
team_url = find_teams_url("http://www.espn.com/nba/teams")
years = [2018]
test = read_teams(team_url, years)

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


http://www.espn.com/nba/team/schedule/_/name/lac/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/min/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/det/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/okc/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/chi/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/utah/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/bos/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/no/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/ny/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/atl/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/phx/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/por/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/cle/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_

In [218]:
print(test['bos/boston-celtics'])
# print(test['bos/boston-celtics'])
# from datetime import datetime
# datetime_object1 = datetime.strptime(test['bos/boston-celtics'][2018][2][0][2], "%a, %b %d")
# datetime_object2 = datetime.strptime(test['bos/boston-celtics'][2018][2][0][4], "%a, %b %d")
# (datetime_object2 - datetime_object1).days

                    0                 1             2
0   Tue, Oct 17, 2017       @ Cleveland       L102-99
1   Wed, Oct 18, 2017      vs Milwaukee      L108-100
2   Fri, Oct 20, 2017    @ Philadelphia       W102-92
3   Tue, Oct 24, 2017       vs New York       W110-89
4   Thu, Oct 26, 2017       @ Milwaukee        W96-89
5   Sat, Oct 28, 2017           @ Miami        W96-90
6   Mon, Oct 30, 2017    vs San Antonio       W108-94
7    Wed, Nov 1, 2017     vs Sacramento       W113-86
8    Fri, Nov 3, 2017   @ Oklahoma City       W101-94
9    Sun, Nov 5, 2017         @ Orlando       W104-88
10   Mon, Nov 6, 2017         @ Atlanta      W110-107
11   Wed, Nov 8, 2017    vs Los Angeles       W107-96
12  Fri, Nov 10, 2017      vs Charlotte        W90-87
13  Sun, Nov 12, 2017        vs Toronto        W95-94
14  Tue, Nov 14, 2017        @ Brooklyn      W109-102
15  Thu, Nov 16, 2017   vs Golden State        W92-88
16  Sat, Nov 18, 2017         @ Atlanta       W110-99
17  Mon, Nov 20, 2017       

In [219]:
train = read_teams(team_url, [2013,2014,2015,2016,2017])

http://www.espn.com/nba/team/schedule/_/name/lac/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/lac/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/lac/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/lac/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/lac/season/2017/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/min/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/min/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/min/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/min/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/min/season/2017/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/det/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/det/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/det/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/

http://www.espn.com/nba/team/schedule/_/name/bkn/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/bkn/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/bkn/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/bkn/season/2017/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/mia/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/mia/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/mia/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/mia/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/mia/season/2017/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/lal/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/lal/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/lal/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/lal/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/

In [220]:
print(train['bos/boston-celtics'])

                     0                 1             2
0    Tue, Oct 30, 2012           @ Miami      L120-107
1     Fri, Nov 2, 2012      vs Milwaukee        L99-88
2     Sat, Nov 3, 2012      @ Washington        W89-86
3     Wed, Nov 7, 2012     vs Washington    W100-94 OT
4     Fri, Nov 9, 2012   vs Philadelphia      L106-100
5    Sat, Nov 10, 2012       @ Milwaukee        W96-92
6    Mon, Nov 12, 2012         @ Chicago       W101-95
7    Wed, Nov 14, 2012           vs Utah        W98-93
8    Thu, Nov 15, 2012        @ Brooklyn       L102-97
9    Sat, Nov 17, 2012        vs Toronto       W107-89
10   Sun, Nov 18, 2012         @ Detroit       L103-83
11   Wed, Nov 21, 2012    vs San Antonio      L112-100
12   Fri, Nov 23, 2012  vs Oklahoma City      W108-100
13   Sun, Nov 25, 2012         @ Orlando   W116-110 OT
14   Wed, Nov 28, 2012       vs Brooklyn        L95-83
15   Fri, Nov 30, 2012       vs Portland        W96-78
16    Sat, Dec 1, 2012       @ Milwaukee        L91-88
17    Wed,

In [233]:
from datetime import datetime
def data_processing(test):
    game_info = {}
    for team in test:
        for i in range(5,len(test[team][1])):
            if team not in game_info:
                game_info[team] = {}
            info = [test[team][1][i][2:]]
            for j in range(1,6):
                if test[team][2][5-j][0] == 'L':
                    info.append('1')
                else:
                    info.append('0')
            if (test[team][1][i][1] == 'v'):
                info.append('1')
            else:
                info.append('0')
            d1 = datetime.strptime(test[team][0][i], "%a, %b %d, %Y")
            d2 = datetime.strptime(test[team][0][i-1], "%a, %b %d, %Y")
            if (d1-d2).days==1:
                info.append('1')
            else:
                info.append('0')
            game_info[team] = info
    return game_info

In [236]:
data_processing(test)['no/new-orleans-pelicans']

[' San Antonio', '0', '1', '0', '1', '1', '0', '0']